# **저시력자를 위한 원화 화폐 분류**
---
- 본 과제는 UltraLytics YOLO v5 모델 사용을 권장합니다.
    - 본 파일의 목차는 UltraLytics YOLO v5에 맞게 작성되어 있습니다.
    - 다른 모델을 찾아서 사용하셔도 좋습니다.
    - 산출물이 잘 나오면 됩니다 : )
---

## 0.미션
---
- **과제 수행 목표**
    - 본 과제는 Object Detection 문제입니다.
    - Object Detection 문제로 접근하기 위해 **데이터셋 전처리**를 하셔야 합니다.
    - 데이터셋 : money_dataset.zip
        1. 데이터셋은 압축 파일로 제공됩니다.
        2. 압축 파일 안에는 화폐마다 폴더가 개별적으로 존재합니다.
        3. 폴더 안에는 화폐 이미지와 화폐 정보가 담긴 json 파일이 있습니다.
    - 여러분이 직접 촬영한 화폐 사진들을 탐지 과정에서 이용 해보세요.
    - 이미지에 화폐 하나만 나오게 촬영하는 것은 지양해주세요.
    - 다양한 방법으로 화폐를 촬영하고 결과를 확인해보세요.
        - ex 1) 화폐의 모든 종류를 한 이미지에 나오게 촬영
        - ex 2) 여러 화폐를 겹치게 하여 촬영
---
- **Key Point**
    1. 모델에 맞는 폴더 구조 확인
    2. 이미지 축소 비율에 맞춰 좌표값 변경
        - 좌표를 이미지 리사이즈한 비율로 변경
    3. 모델에 맞는 정보 추출/형식 변경
        - json 파일에서 정보 추출 및 모델 형식에 맞게 변경
    4. 화폐당 하나의 클래스로 변경
        - 총 8개 클래스
    5. 모델 선택 필요
---

## 1.환경설정

### (1) 구글 드라이브 연동, 데이터 다운로드
---
- 아래의 코드 셀을 반드시 실행시켜야 합니다.
---

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
!pip install gdown

### (2) 데이터셋 불러오기
---
- **세부요구사항**
    - 데이터셋 파일의 압축을 해제하세요.
---
- 예제 코드에서는 zipfile 모듈을 이용하였습니다.
    - [zipfile document](https://docs.python.org/3/library/zipfile.html#zipfile-objects)
    - 해당 모듈 이외에 자신이 잘 알고 있는 방법을 사용해도 됩니다.
---

In [3]:
import zipfile, gdown,os

In [4]:
import zipfile, gdown,os
url ="https://drive.google.com/file/d/1k1tXDK35s6BsMTPGWSl5GVGNoPfC898X/view?usp=drive_link"
file_name = "money_dataset.zip"
output = "/content/drive/MyDrive/" + file_name # 변경 가능
if not os.path.exists(output):
    gdown.download(url=url, output=output, quiet=False, fuzzy=True)

In [5]:
# 데이터셋 압축 파일 경로 : 유저별로 상이할 수 있음
money_data = zipfile.ZipFile(output)

In [6]:
# 데이터셋 압축 해제
money_data.extractall('/content/Dataset/')

## 2.데이터 전처리

### (1) 폴더 구조 생성 및 파일 이동
---
- **세부요구사항**
    -  모델에서 요구하는 폴더 구조를 만들어야 합니다.
        - Hint : Image와 Label을 구분하는 폴더를 만들어 주세요
---
- 예제 코드에서는 glob, shutil 모듈을 이용하였습니다.
    - [glob document](https://docs.python.org/3/library/glob.html) | [shutil document](https://docs.python.org/3/library/shutil.html)
    - 해당 모듈 이외에 자신이 잘 알고 있는 방법을 사용해도 됩니다.
---

In [7]:
# 1.폴더 구조 만들기
!mkdir /content/Dataset/images;
!mkdir /content/Dataset/images/train; mkdir /content/Dataset/images/val

!mkdir /content/Dataset/labels;
!mkdir /content/Dataset/labels/train; mkdir /content/Dataset/labels/val

In [8]:
import glob, shutil
from sklearn.model_selection import train_test_split
import cv2
import json
import numpy as np

In [9]:
# 2. Dataset metadata 입력
won_list = [ '10', '50', '100', '500', '1000', '5000', '10000', '50000']
data_path = '/content/Dataset/'

---
- 데이터를 Training set | Validation set으로 분할하세요.
    - 예시 : Training과 Validation은 8:2로 분리
- Hint : 이미지 데이터는 /images에, JSON 데이터는 /labels에 넣어주세요
    - 예시 : /dataset/images/train, /dataset/labels/train
    - 예제 코드에서는 glob, shutil 모듈을 이용하였습니다.
    - [glob document](https://docs.python.org/3/library/glob.html) | [shutil document](https://docs.python.org/3/library/shutil.html)

    ※ 해당 모듈 이외에 자신이 잘 알고 있는 방법을 사용해도 됩니다.
    
---

In [ ]:
########################
# 이 셀부터 코드 작성하세요
########################
# 3. 데이터를 Training set | Validation set으로 분할하세요.

# 디렉토리 내에서 확장자가 .img인 파일을 찾음
def read_json(json_path):
    with open(json_path, 'r') as f:
        json_file = json.load(f)
    return json_file

def move_files(file_list, src_path, dst_path_train, dst_path_val):
    for file in file_list:
        print(file, 'file 입니다')
        # /content/Dataset/100/100_1000_1.jpg
        image = file.split('/')[-1] + '.jpg'
        json = file.split('/')[-1] + '.json'
        rotated_image = file.split('/')[-1] + '_rotated.jpg'
        rotated_json = file.split('/')[-1] + '_rotated.json'


        shutil.copy(os.path.join(src_path ,image), dst_path_train)
        shutil.copy(os.path.join(src_path, json), dst_path_val)

        shutil.copy(os.path.join(src_path ,rotated_image), dst_path_train)
        shutil.copy(os.path.join(src_path, rotated_json), dst_path_val)

def rotate_image_and_box(image_path, json_file, angle=90):
    # 이미지 읽기
    image = cv2.imread(image_path)
    print(image)
    h, w = image.shape[:2]


    # 이미지 중심점을 기준으로 90도 회전
    M = cv2.getRotationMatrix2D((w//2, h//2), angle, 1)
    rotated_image = cv2.warpAffine(image, M, (w, h))

    # bounding box 좌표도 회전
    shape = json_file['shapes'][0]
    x1, y1 = shape['points'][0]
    x2, y2 = shape['points'][1]

    box_center = np.array([[(x1 + x2) / 2, (y1 + y2) / 2]], dtype='float32')
    box_center = np.column_stack((box_center, np.ones(box_center.shape[0], dtype='float32')))

    rotated_center = np.dot(M, box_center.T).T
    rotated_x, rotated_y = rotated_center[0][:2]

    # 회전된 bounding box의 새로운 좌표 계산
    new_x1, new_y1 = rotated_x - abs(x1 - x2) / 2, rotated_y - abs(y1 - y2) / 2
    new_x2, new_y2 = rotated_x + abs(x1 - x2) / 2, rotated_y + abs(y1 - y2) / 2

    # 회전된 이미지와 bounding box 좌표 반환
    return rotated_image, [[new_x1, new_y1], [new_x2, new_y2]]

In [ ]:
# 지폐 선택 - 10, 100, 500, ...
for won in won_list:
    # src 경로 설정
    src_path = data_path + won + '/'

    # 저장할 파일 설정
    train_images = '/content/Dataset/images/train'
    val_imgaes = '/content/Dataset/images/val'
    train_json = '/content/Dataset/labels/train'
    val_json = '/content/Dataset/labels/val'

    # 데이터를 나누기 위해 파일 이름 불러오기
    image_files = glob.glob(os.path.join(src_path, '*.jpg'))
    images = [name.replace('.jpg', '') for name in image_files]

    for img in image_files:
        image_path = img
        json_path = image_path.replace('.jpg', '.json') # 파일명
        json_file = read_json(json_path) # 파일을 json 으로 읽음
        rotated_image, rotated_box =  rotate_image_and_box(image_path, json_file)

        # rotated_image_filename = os.path.splittext(file)[0] + '_rotated.jpg'
        rotated_image_filename = image_path.replace('.jpg', '_rotated.jpg')
        cv2.imwrite(os.path.join(src_path, rotated_image_filename), rotated_image)


        json_file['shapes'][0]['points'] = rotated_box
        json_file['imageWidth'], json_file['imageHeight'] = json_file['imageHeight'], json_file['imageWidth']
        # rotated_json_filename = os.path.splitext(file)[0] + '_rotated.json'
        rotated_json_path = json_path.replace('.json', '_rotated.json')
        # print(rotated_json_path)
        with open(rotated_json_path, 'w') as f:
            json.dump(json_file, f)

    # 데이터 나누기
    train_names, val_names = train_test_split(images, test_size=0.2,random_state=2023, shuffle=True)
    move_files(train_names, src_path, train_images, train_json)
    move_files(val_names, src_path, val_imgaes, val_json)

yolo5/detect.py의 default 파라미터를 조정해서 학습할 수 있음.

### (2) json에서 정보 추출
---
- **세부요구사항**
    - json 파일에서 필요한 정보를 추출하세요:
        - 위치 정보 : x1, x2, y1, y2
        - 박스 정보 : shape_type
        - 클래스 정보 : labels
    - 화폐당 하나의 클래스로 변경하세요.
        - json 파일에는 화폐 클래스가 앞뒷면으로 구분되어 있습니다.
        - 화폐의 앞뒷면 구분을 없애주세요.
            - 예시 : 'ten_front', 'ten_back' -> 'ten'
    - 화폐의 위치 정보를 YOLO 모델 형식에 맞게 변경 해주세요.
        - 사용되는 이미지는 원본에서 1/5로 축소되어 있습니다.
        - json 파일의 정보는 원본 기준 데이터이므로 위치 정보 추출을 할 때 x값과 y값을 1/5로 줄여주세요.
    - 이렇게 변경된 정보를 YOLO label 형식에 맞게 txt파일로 저장 해 주세요.
        - Hint : YOLO Labeling Format [label, x-center, y-center, width-norm, height-norm]
---

In [ ]:
import os, json

In [ ]:
json_path = '/content/Dataset/labels/'
temp_list = ['train', 'val']

In [ ]:
########################
# 이 셀부터 코드 작성하세요
# Json 파일에서 필요한 정보만 골라 txt로 바꾸는 작업임을 기억하세요!
########################
money_class = ['Ten' ,'Fifty', 'Hundred', 'Five_Hundred', 'Thousand', 'Five_Thousand', 'Ten_Thousand', 'Fifty_Thousand']


result =  ''

# 데이터를 나누기 위해 파일 이름 불러오기
for end in temp_list:
    image_files = glob.glob(os.path.join(json_path, end, '*.json'))
    print(len(image_files))
    count = 0
    for file in image_files:
        text = file.replace('.json', '')
        with open(file, 'r') as json_file:
            data = json.load(json_file)

        # 파일 이름
        text_name = text.split("/")[-1]
        # 라벨
        tmp = '_'.join(data['shapes'][0]['label'].split('_')[:-1])
        label = money_class.index(tmp)

        # x1
        x1 = data['shapes'][0]['points'][0][0]
        # y1
        y1 = data['shapes'][0]['points'][0][1]
        # x2
        x2 = data['shapes'][0]['points'][1][0]
        # y2
        y2 = data['shapes'][0]['points'][1][1]

        # ih
        ih = data['imageHeight'] / 5
        # iW
        iw = data['imageWidth'] / 5


        X = ((x1+x2)/2) / 5 / iw
        Y = ((y1+y2)/2) / 5 / ih

        # H
        H = (y2- y1) /5 / ih
        # W
        W = (x2-x1) / 5 / iw

        tmp = [label, X, Y, W, H]

        result = ' '.join(map(str, tmp))

        # 파일 경로 설정 (원하는 경로와 파일명을 지정하세요)
        file_path = data_path + 'labels/' + end +'/'+ text_name + '.txt'

        # 텍스트 파일로 문자열 저장
        with open(file_path, "w", encoding="utf-8") as text_file:
            text_file.write(result)
        os.remove(file)

        count += 1
    print(count)



8344
8344
2092
2092


### (3) 데이터셋 정보가 담긴 파일 생성
---
- **세부요구사항**
    - 파일 안에 있어야 할 정보는 아래와 같습니다.
        - 학습할 클래스 이름 정보
        - 학습할 클래스 수 정보
        - Training, Validation 데이터셋 위치 정보
---
- 가장 대중적으로 이용하는 라이브러리는 yaml 입니다.
    - [yaml document](https://pyyaml.org/wiki/PyYAMLDocumentation)
    - 해당 모듈 이외에 자신이 잘 알고 있는 방법을 사용해도 됩니다.
---

In [ ]:
import yaml

In [ ]:
won_dict = {0:'10', 1:'50', 2:'100', 3:'500', 4:'1000', 5:'5000', 6:'10000', 7:'50000'}


In [ ]:
########################
# 이 셀부터 코드 작성하세요
########################
import yaml

yaml_data = {"names": won_dict,
             "nc":8,
             "train": '/content/Dataset/images/train',
             "val": '/content/Dataset/images/val'
             }

with open('/content/Dataset/money.yaml', "w") as f:
  yaml.dump(yaml_data, f)

## 3.모델링

### (1) 모델 라이브러리 설치
---

yolov5가 stable한 버전이라 v5를 사용하겠다. docs에도 매우 자세히 나와있음.

https://docs.ultralytics.com/yolov5/

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
!pip install -r yolov5/requirements.txt  # install

Cloning into 'yolov5'...
remote: Enumerating objects: 15994, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 15994 (delta 18), reused 18 (delta 12), pack-reused 15967
Receiving objects: 100% (15994/15994), 14.64 MiB | 18.09 MiB/s, done.
Resolving deltas: 100% (10980/10980), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.0/190.0 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 618.4/618.4 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.6 MB/s eta 0:00:00


### (2) 가중치 파일 다운로드
---
- **세부요구사항**
    - 모델 개발자가 제공하는 사전 학습 가중치 파일을 다운로드 하세요.
        - 해당 과정이 불필요하다면 넘어가셔도 됩니다!
---

In [ ]:
# Freeze Backbone - 학습 속도 향상
# !python train.py --freeze 10

In [ ]:
########################
# 이 셀부터 코드 작성하세요
########################

### (3) 학습 : train.py
---
- **세부요구사항**
    - UltraLytics YOLO v5에는 아래의 데이터가 필요합니다.
        - 데이터셋 정보가 담긴 yaml 파일
        - 사용하려는 모델 구조에 대한 yaml 파일
        - 사용하려는 모델의 가중치 파일
---

In [ ]:
%cd yolov5

/content/yolov5


In [ ]:
########################
# 이 셀부터 코드 작성하세요
########################
# !python train.py --batch-size 16 --epochs 100 --data  /content/Dataset/money.yaml --cfg /content/yolov5/models/yolov5n.yaml
!python train.py --batch 48 --weights /content/drive/MyDrive/mp3/best.pt --data /content/Dataset/money.yaml --epochs 10 --cache --img 640 --freeze 10

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
train: WARNING ⚠️ /content/Dataset/images/train/10000_B_STUFF_0_56_rotated.jpg: ignoring corrupt image/label: negative label values [   -0.26532]
train: WARNING ⚠️ /content/Dataset/images/train/10000_B_STUFF_0_57_rotated.jpg: ignoring corrupt image/label: negative label values [   -0.31227]
train: WARNING ⚠️ /content/Dataset/images/train/10000_B_STUFF_0_58_rotated.jpg: ignoring corrupt image/label: negative label values [   -0.29284]
train: WARNING ⚠️ /content/Dataset/images/train/10000_B_STUFF_0_59_rotated.jpg: ignoring corrupt image/label: negative label values [    -0.3135]
train: WARNING ⚠️ /content/Dataset/images/train/10000_B_STUFF_0_5_rotated.jpg: ignoring corrupt image/label: negative label values [   -0.24814]
train: WARNING ⚠️ /content/Dataset/images/train/10000_B_STUFF_0_61_rotated.jpg: ignoring corrupt image/label: negative label values [   -0.17353]
train: WARNING ⚠️ /content/Dataset/images/train/10000_B_STUFF_0_62_rotated.jpg: ignoring 

## 4.탐지 : detect.py
---
- **세부요구사항**
    - 학습 과정에서 생성된 가중치 파일을 이용하세요.
    - IoU threshold를 0.25 이하로 설정하세요.
    - confidence threshold를 0.75 이상으로 설정하세요.
---
- 여러분이 **직접 촬영한 화폐 사진과 동영상**을 탐지 과정에 이용하여 결과를 확인하세요.
    - 조건
        1. 화폐의 수를 늘려가며 촬영 해보세요.
            - ex) 50원 하나, 50원 둘, 50원 셋, ...
        2. 화폐의 종류를 늘려가며 촬영 해보세요.
            - ex) 50원 하나와 100원 하나, 50원 하나와 100원 하나와 1000원 하나, ...
        3. 사진은 최소 30장 이상, 동영상은 최소 하나 이상 촬영하여 사용 해보세요.
---

In [ ]:
# test
# python val.py --weights yolov5x.pt --data coco.yaml --img 640 --half

In [ ]:
# 기본 사용
# python detect.py --weights yolov5s.pt --source 0                               # webcam
#                                                img.jpg                         # image
#                                                vid.mp4                         # video
#                                                screen                          # screenshot
#                                                path/                           # directory
#                                                list.txt                        # list of images
#                                                list.streams                    # list of streams
#                                                'path/*.jpg'                    # glob
#                                                'https://youtu.be/LNwODJXcvt4'  # YouTube
#                                                'rtsp://example.com/media.mp4'  # RTSP, RTMP, HTTP stream

!python /content/yolov5/detect.py --weights /content/yolov5/runs/train/exp2/weights/best.pt --conf 0.75 --iou-thres 0.25  --source /content/drive/MyDrive/mp3/KakaoTalk_20230922_135042395.mp4



detect: weights=['/content/yolov5/runs/train/exp2/weights/best.pt'], source=/content/drive/MyDrive/mp3/KakaoTalk_20230922_135042395.mp4, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.75, iou_thres=0.25, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-224-g6262c7f Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)

Fusing layers... 
Model summary: 322 layers, 86220517 parameters, 0 gradients, 203.9 GFLOPs
video 1/1 (1/425) /content/drive/MyDrive/mp3/KakaoTalk_20230922_135042395.mp4: 640x384 (no detections), 44.7ms
video 1/1 (2/425) /content/drive/MyDrive/mp3/KakaoTalk_20230922_135042395.mp4: 640x384 (no detections), 14.0ms
video 1/1 (3/425) /content/drive/MyDr

In [ ]:
shutil.move('/content/yolov5/runs/detect/exp2', '/content/drive/MyDrive/mp3')

'/content/drive/MyDrive/mp3/exp2'

In [ ]:
## 소스 파일
#!python detect.py --weights best.pt --source path/to/images

In [ ]:
########################
# 이 셀부터 코드 작성하세요
########################
!python detect.py --weights /content/yolov5/runs/train/exp/weights/best.pt --img-size 640 --conf 0.4 --source /content/money_data/KakaoTalk_20230921_163159696.jpg

## 5. 실시간 데모

- Ultralytics HUB App

- NNAPI

- Android | Pytorch

- deeplearning model for mobile

In [ ]:
!pip install ultralytics

In [11]:
from ultralytics import YOLO
cap = cv2.VideoCapture(2)

model = YOLO(model='/content/drive/MyDrive/mp3/best.pt', task='detect')
while True:
  ret, img = cap.read()
  result = model.predict(source=img, conf=0.25, iou=0.7, save=False,
                         line_width=2, verbose=False)
  img = result[0].plot()

  cv2.imshow('Bill Detection', img)

  if cv2.waitKey(10) == ord('q'):
    break
cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: ignored

###모델 변환

1.PyTorch에서 ONNX로 변환

In [ ]:
!pip uninstall torch torchvision
!pip install torch torchvision

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
!pip install -r yolov5/requirements.txt  # install

In [9]:
%cd yolov5

/content/yolov5


In [ ]:
import sys

# YOLOv5 디렉토리 경로를 sys.path에 추가
sys.path.append('./yolov5/')

In [19]:
!pip install onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 68.0 MB/s eta 0:00:00


In [20]:
# 필요한 모듈들을 임포트
import torch
from models.experimental import attempt_load  # attempt_load 함수 임포트

# 모델 로드
model_path = '/content/drive/MyDrive/mp3/best.pt'
model = attempt_load(model_path)
model = model.to('cuda')  # 모델을 GPU로 이동
model.eval()  # 평가 모드로 설정

dummy_input = torch.randn(1, 3, 416, 416).to('cuda')  # Assuming model takes 416x416 RGB images and move it to GPU

model = model.cpu()
dummy_input = dummy_input.cpu()

torch.onnx.export(model, dummy_input, "model.onnx")

Fusing layers... 
Model summary: 322 layers, 86220517 parameters, 0 gradients, 203.9 GFLOPs
/content/yolov5/models/yolo.py:207: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if augment:
/content/yolov5/models/yolo.py:119: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if profile:
/content/yolov5/models/yolo.py:123: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not gener

============= Diagnostic Run torch.onnx.export version 2.0.1+cu117 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



2.ONNX에서 CoreML로 변환

In [ ]:
!pip install coremltools

In [24]:
!pip install onnx-coreml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for typing: filename=typing-3.7.4.3-py3-none-any.whl size=26305 sha256=fc97569e576eeb87e5c5562255c1005b78a6fad460e40d61d06eb5bf47230ef6
  Stored in directory: /root/.cache/pip/wheels/7c/d0/9e/1f26ebb66d9e1732e4098bc5a6c2d91f6c9a529838f0284890
Successfully built typing


In [6]:
import coremltools
from onnx_coreml import convert

coreml_model = convert(model='model.onnx',
                       mode='classifier',  # or 'regressor' based on your task
                       image_input_names=['image'],
                       preprocessing_args={'image_scale': 1/255.0})
coreml_model.save('model.mlmodel')

ModuleNotFoundError: ignored